## Basic Data Processing Pipline
Author:
- Beitong Tian - beitong2@illinois.edu

Nov. 2022

Raw Dataset Location: `/data4/dataset`

(Please use a test dataset. E.g. copy part of the dataset to another place for experiment)

Some Assumptions:
1. Sensory data is stored as files in the dataset
2. All files have the same format (sampling rate, bit width, ...)

Features: 
1. Read all filepaths
2. Print dataset stats (number of files, sampling rate, sample width, duration of each file, etc)
3. Check duplicate filenames
4. Plot Data Availablity graph (show missing timestamps)
5. Iterate Dataset with WindowSize and StepSize
6. Extract features of each window
7. Train an auto encoder with features
8. Test the model on remaining dataset

9. Generate and augment anomaly in dataset
10. Insert anomalies into the dataset

11. Print results of the model on the labeled dataset

Visualizations:
1. view a window
2. view a file
3. view files
4. view the dataset
5. view data around a suspicious anomaly score
6. view model performance (confusion matrix, auc plot)
7. view anomaly results (anomaly over original graph, paint the abnormal window in light right)

### Import helper functions

In [ ]:
%load_ext autoreload
%autoreload 2
from common import *
from visualizationCommon import *
from feature import *
from modelCommon import *

### Set Metadata

In [ ]:
# Please use a test dataset
dataDirs = ['/data4/anomalyDetectionTest/experiment3AnomalyDetection'] # the dataset may be stored in multiple dirs, here we use a list to store all dirs
windowSize = 48000
stepSize = 24000

### Feature 1: Read all filepaths
1. Filepath is the full path of a file
2. Filename is the name of a file

In [ ]:
filepaths = getAllFilePaths(dataDirs)
filePathsByday = getAllFilePathByDay(filepaths)
printFilePathByDay(filePathsByday)

### Feature 2: Print dataset/file stats

In [ ]:
# dataset stats
datasetStat = getDatasetStat(filepaths)
print(datasetStat)

In [ ]:
# file stats
filepath = filepaths[0]
fileStat = getFileStat(filepath)
print(fileStat)

### Feature 3: Check Duplicate Files

In [ ]:
with MaintletTimer("Feature 3: check duplicate files") as mt:
    duplicates, isDuplicate = checkDuplicates(filepaths)
    print("Duplicate List:")
    for duplicate in duplicates:
        print(duplicate)
    print(f"isDuplicate: {isDuplicate}")

### Feature 4: Plot Data Availablity graph (show missing files along the time axis)

In [ ]:
# convert filepath to filetime
# round filetime to seconds
# check duplicate
x = []
for filepath in filepaths:
    baseTime = pathToTime(filepath)
    for t in range(int(fileStat.duration)):
        x.append(baseTime)
        baseTime += timedelta(seconds=1)

with MaintletTimer("Check duplicate file timestamps (the granularity is second)") as mt:
    duplicates, isDuplicate = checkDuplicates(x)
    print("Duplicate List:")
    for duplicate in duplicates:
        print(duplicate)
    print(f"isDuplicate: {isDuplicate}")

In [ ]:
# create X axis: all timestamps (in seconds) from the beginning to the end of recording
# create Y axis: 1 for available, 0 for not available
startTime = x[0]
endTime = x[-1]
X = [pd.to_datetime(d) for d in pd.date_range(start=startTime, end=endTime, freq='S')]
Y = []
missingTimestamps = []
lX = 0
lx = 0
while lx < len(x) and lX < len(X):
    if x[lx].date() == X[lX].date():
        Y.append(1)
        lx+=1
        lX+=1
    else:
        Y.append(0)
        missingTimestamps.append(X[lX])
        lX+=1


In [ ]:
if len(missingTimestamps) > 0:
    start = missingTimestamps[0]
    end = missingTimestamps[0]
    missingTimeRanges = []
    for i in range(len(missingTimestamps)-1):
        currentT = missingTimestamps[i+1]
        previousT = missingTimestamps[i]
        deltaT = (currentT - previousT).total_seconds()

        if deltaT > 1.0:
            missingTimeRanges.append([start, end])
            start = currentT
        else:
            end = currentT
    missingTimeRanges.append([start, missingTimestamps[-1]])
    print("Missing Time Range")
    for missingTimeRange in missingTimeRanges:
        print(f"start {missingTimeRange[0]} end {missingTimeRange[1]}")

In [ ]:
%matplotlib widget
f, ax = plt.subplots(figsize=(10, 6))
f.suptitle('Data Availablity Plot', fontsize=20)
ax.plot(X, Y)
ax.set_yticks([0,1])
ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
ax.set_xlabel("Time")
ax.set_ylabel("Availability")
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
plt.tight_layout()
plt.show()

### Feature5: Iterate Dataset with settings of WindowSize and StepSize

In [ ]:
filepaths = getAllFilePaths(dataDirs)
# here we only use the first two files to demo the feature
filepaths = filepaths[:2]
windowIteratorClass = WindowIterator(filepaths, windowSize, stepSize)
windowIter = iter(windowIteratorClass)
print(f"Now Iterating Data Set with Window Size {windowSize} ({windowSize/fileStat.sr} Seconds) and Step Size {stepSize} ({round(stepSize/fileStat.sr,2)} Seconds)\n")
for window, sampleIndex, windowIndex, time, worldTime in windowIter:
    print(f"Window shape      : {window.shape}")
    print(f"SampleIndex       : {sampleIndex}")
    print(f"WindowIndex       : {windowIndex}")
    print(f"Start Time        : {time}")
    print(f"Start World Time  : {worldTime[0]}")
    print(f"End World Time    : {worldTime[1]}")
    print()

### Visualization 1: View Window

In [ ]:
%matplotlib inline
sensor = 0
filepaths = getAllFilePaths(dataDirs)
# here we only use the first file to demo the feature
filepaths = filepaths[:1]
windowIteratorClass = WindowIterator(filepaths, windowSize, stepSize)
windowIter = iter(windowIteratorClass)
print(f"Now Iterating Data Set with Window Size {windowSize} ({windowSize/fileStat.sr} Seconds) and Step Size {stepSize} ({round(stepSize/fileStat.sr,2)} Seconds)\n")
for window, sampleIndex, windowIndex, time, worldTime in windowIter:
    visualizeWindow(window[:, sensor])
    print(f"Window shape      : {window.shape}")
    print(f"SampleIndex       : {sampleIndex}")
    print(f"WindowIndex       : {windowIndex}")
    print(f"Start Time        : {time}")
    print(f"Start World Time  : {worldTime[0]}")
    print(f"End World Time    : {worldTime[1]}")
    print()


### Visualization 2-3: View Files

In [ ]:
%matplotlib inline
filepaths = getAllFilePaths(dataDirs)
# here we only use the first two files to demo the feature
filepaths = filepaths[:2]
visualizeFiles(filepaths)

### Visualization 4: View Dataset

In [ ]:
%matplotlib inline
filepaths = getAllFilePaths(dataDirs)
# here we only use the first 10 files to demo the feature
filepaths = filepaths[:10]
visualizeDataset(filepaths, chunkSize = 5)

### Feature 6: Extract features of each window (Log Mel Spectrogram Energy)

In [ ]:
# Training Dataset: first 100 files
filepaths = getAllFilePaths(dataDirs)
filepaths = filepaths[:100]

# extract features 
sensorIndex = 0
totalWindowCount = getTotalWindowCount(len(filepaths), fileStat.sampleCount, windowSize, stepSize)
data = [] # features
windows = []
n_mels = 64
n_frames = 1
dims = n_mels * n_frames

windowIteratorClass = WindowIterator(filepaths, windowSize, stepSize)
windowIter = iter(windowIteratorClass)
print(f"Now Iterating Data Set with Window Size {windowSize} ({windowSize/fileStat.sr} Seconds) and Step Size {stepSize} ({round(stepSize/fileStat.sr,2)} Seconds)\n")

for window, sampleIndex, windowIndex, _time, worldTime in windowIter:
    # get features
    vectors = feature1_LogMelEnergies(window[:,sensorIndex])
    
    # concat features to dataset
    if windowIndex == 0:
        data = np.zeros((totalWindowCount * vectors.shape[0], dims), float)
    data[vectors.shape[0] * windowIndex : vectors.shape[0] * (windowIndex + 1), :] = vectors
    


### Feature 7: Train an auto encoder with features

In [ ]:
import keras_model
# set some parameters for the model
lr = 0.001
epochs = 100
batch_size = 512
shuffle = True
validation_split = 0.1
verbose = 1
model = keras_model.get_model(n_mels * n_frames, lr)

In [ ]:
model.summary()

In [ ]:
# train the model
history = model.fit(x=data,
                    y=data,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=shuffle,
                    validation_split=validation_split,
                    verbose=verbose)

In [ ]:
# save the model
saveModel("autoencoder", model)

In [ ]:
# plot the train loss
loss_plot(history.history["loss"], history.history["val_loss"])

In [ ]:
# calculate and visualize the anomaly score in the training dataset
# ref: https://github.com/Kota-Dohi/dcase2022_task2_baseline_ae/blob/main/00_train.py
y_pred = []
start_idx = 0
n_vectors_ea_file = int(data.shape[0] / totalWindowCount)
for file_idx in range(totalWindowCount):
    predict = model.predict(data[start_idx : start_idx + n_vectors_ea_file, :])
    
    original = data[start_idx : start_idx + n_vectors_ea_file, :]
    
    # plot spectrogram to visulize the difference between the original and generated spectrogram
    # print(file_idx)
    # plotSpectrogram(original)
    # plotSpectrogram(predict)
    
    y_pred.append(np.mean(np.square(original - predict)))
    start_idx += n_vectors_ea_file
    
plt.plot(y_pred)
plt.title("anomaly score in the training dataset")
plt.show()

In [ ]:
# fit anomaly score distribution
shape_hat, loc_hat, scale_hat = fitAnomalyScore(y_pred)
# calculate anomaly score threshold
decision_threshold = scipy.stats.gamma.ppf(q=0.9, a=shape_hat, loc=loc_hat, scale=scale_hat)
print(f"Anomaly Threshold: {decision_threshold}")

### Feature 8: Test the model on remaining dataset

In [ ]:
from IPython import display
# the flag for enabling the animation
isAnimate = False

# test dataset: here we use the following 100 files 
filepaths = getAllFilePaths(dataDirs)
filepaths = filepaths[100:200]

# setup parameters
sensorIndex = 0
newPreds = []
avgTime = 0
counter = 0
plotSampleCount = 10000
windowTimeBuffer = [] # record the world time for each window
x = [i-plotSampleCount for i in range(plotSampleCount)]
y = []

# animation
plt.figure(figsize=(18, 6))
def animate(newData, startIndex):
    global line, x, y, plotSampleCount
    x = x[1:]
    x.append(startIndex)
    if len(y) == 0:
        y = [newData for i in range(plotSampleCount)]
    else:
        y = y[1:]
        y = np.append(y, newData)
    # line.set_xdata(x)
    # line.set_ydata(y)  # update the data
    plt.gca().cla() 
    plt.plot(x,y)
    display.clear_output(wait=True)
    display.display(pl.gcf())

# iterate over test dataset and get predicts
totalWindowEstimate =  getTotalWindowCount(len(filepaths), fileStat.sampleCount, windowSize, stepSize)
windowIteratorClass = WindowIterator(filepaths, windowSize, stepSize)
windowIter = iter(windowIteratorClass)
print(f"Now Iterating Data Set with Window Size {windowSize} ({windowSize/fileStat.sr} Seconds) and Step Size {stepSize} ({round(stepSize/fileStat.sr,2)} Seconds)\n")
for window, sampleIndex, windowIndex, timeIndex, worldTime in windowIter:
    windowTimeBuffer.append(worldTime)
    startTime = time.time()
    # window shape: sample x channel
    vectors = feature1_LogMelEnergies(window[:,sensorIndex])
    predict = model.predict(vectors)
    
    original = vectors
    # print(file_idx)
    # plotSpectrogram(original)
    # plotSpectrogram(predict)
    newAbnormalScore = np.mean(np.square(original - predict))
    if isAnimate:
        animate(newAbnormalScore, windowIndex)
    # append abnormal scores
    newPreds.append(newAbnormalScore)
    
    # make decision (TODO 11072022)
    # idea1: just compare the abnormal score with the threshold (either set the p to 0.999 which is nonsense, or generate lots of FPs)
    # idea2: build a probability model to reduce the FP
    # need to read more papers
    
    
    # calculate inference time
    elapsedTime = time.time() - startTime
    if avgTime == 0:
        avgTime = elapsedTime
    else:
        avgTime = (avgTime*counter + elapsedTime) / (counter+1)
    counter += 1
    print(f"\r\ravgTime {round(avgTime*1000, 4): <10} ms | window: {windowIndex: < 10} out of {totalWindowEstimate: < 10} | Remaining Time {round((totalWindowEstimate - windowIndex)*avgTime, 2)} Seconds", end="",flush=True)
plt.show()

In [ ]:
# plot anomaly scroe results
%matplotlib widget
plt.plot(newPreds)
plt.title("anomaly score from test dataset")
plt.show()

### Visualization 5: view data around an interesting anomaly score

In [ ]:
# step 1: zoom in the plot of anomaly score from test dataset (the plot in the above cell)
# step 2: record the index
# step 3: change the interesting Index below to the index you find in the step 2
%matplotlib inline
interestingIndex = 1880 
checkDataAroundTimeWindow(windowTimeBuffer[interestingIndex], filepaths)

### Feature 11: Print performance of the model on the test dataset

In [ ]:
# here we generate the ground truth and the anomaly detection result
# we consider two kinds of results
# 1. window-wise (1S)
# 2. file-wise (10S)
# also, we need to think about the correct metrics for our problem
# 1. we want to know the start and end of the anomaly so we can clip it. 
# 2. another way is we can choose a long enough window
# is it useful to know a 1 second window is abnormal is useful? 
# we can send a longer suspicious window to the expert and then clip afterwards

# for simulating groundtruth
testFileCount = 100
filepaths = range(testFileCount)
totalWindowCount = getTotalWindowCount(len(filepaths), fileStat.sampleCount, windowSize, stepSize)

samples = stats.gamma.rvs(a=shape_hat, scale=scale_hat, loc=loc_hat, size = totalWindowCount) # sample pred from gamma distribution

decisionThresholdForGroundTruth = scipy.stats.gamma.ppf(q=0.91, a=shape_hat, loc=loc_hat, scale=scale_hat)
decisionThresholdForDecision = scipy.stats.gamma.ppf(q=0.9, a=shape_hat, loc=loc_hat, scale=scale_hat)

# replace the following three lines based on the output of your model
yTrue = [1 if i > decisionThresholdForGroundTruth else 0 for i in samples]
yPred = samples # pred score
decision = [1 if i > decisionThresholdForDecision else 0 for i in samples]

max_fpr = 0.1 # for pAUC

In [ ]:
[auc, p_auc, prec, recall, f1] = getModelPerformance(yTrue, yPred, decision, max_fpr)

In [ ]:
# save to csv
csvLines = []
csvLines.append(["model", "AUC", "pAUC", "precision", "recall", "F1 score"])
csvLines.append(["autoencoder", auc, p_auc, prec, recall, f1])
saveCSV("modelRes", csvLines)

### Visualization 6: view model performance (confusion matrix, ROC plot)

In [ ]:
%matplotlib widget
rocPlot(yTrue, yPred)

In [ ]:
confusionMatrixPlot(yTrue, decision)

### End

### Below are some functions only for this script

In [ ]:
# get world time of all windows from file #100
filepaths = getAllFilePaths(dataDirs)[100:]
totalWindowEstimate =  int((len(filepaths)* fileStat.sampleCount - windowSize)/stepSize) + 1
data = []
windows = []
worldTimes = []
n_mels = 64
n_frames = 1
dims = n_mels * n_frames
windowIteratorClass = WindowIterator(filepaths, windowSize, stepSize)
windowIter = iter(windowIteratorClass)
print(f"Now Iterating Data Set with Window Size {windowSize} ({windowSize/fileStat.sr} Seconds) and Step Size {stepSize} ({round(stepSize/fileStat.sr,2)} Seconds)\n")
for window, sampleIndex, windowIndex, time, worldTime in windowIter:
    # window shape: sample x channel
    # windows.append(window)
    worldTimes.append(worldTime)
    if windowIndex % 100 == 0:
        print(f"\rwindow: {windowIndex: < 10} out of {totalWindowEstimate: < 10}", end="",flush=True)


In [ ]:
# load history data
newPreds = loadFile('2022-11-06-16-25-32-newPreds100end')

In [ ]:
print(len(newPreds))

In [ ]:
filepaths = getAllFilePaths(dataDirs)[100:]
len(filepaths)

In [ ]:
totalWindowEstimate =  int((len(filepaths)* fileStat.sampleCount - windowSize)/stepSize) + 1
totalWindowEstimate

In [ ]:
predForEachFile = chunks(newPreds, 20)

In [ ]:
%matplotlib widget
yPredForEachFile = []
for i in predForEachFile:
    yPredForEachFile.append(np.average(i))
print(len(yPredForEachFile))
plt.figure(figsize=(8, 8))
plt.plot(yPredForEachFile)
plt.show()

In [ ]:
%matplotlib inline
visualizeFiles(filepaths[28600:28800])

In [ ]:
visualizeFiles(filepaths[21063:21067], play=True)

In [ ]:
visualizeFiles(filepaths[15195:15200], play=True)